Instructions on How to Use Data Analytics Script:

    1. Click on the "Cell" button above in the toolbar and then click on "Run all" . The code should disappear when you run the second cell, and can be toggled to appear or not with the button reading "Click here to toggle on/off the raw code."
    
    2. Use the interactive dashboard that appears to specify your search parameters. 
        NOTE: Hit Cmd button+Click to select multiple options for the scrollable menu items
        
    3. When search parameters have been specified click "Generate Graph" to generate graph
    

In [4]:
#data analysis code
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

import pandas as pd
import glob

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, Layout, Button, Box
from IPython.display import display, clear_output

###HANDLES INTERACTION BETWEEN INTERFACE AND DATA ANALYSIS CODE AFTER BUTTON ###
def on_button_clicked(run_button):
    clear_output(wait=True)
    display(param_options)
    display(run_button)
  # initializes input into data analysis code from interface selections
    YEARS = years_widget.value
    QUARTERS = quarters_widget.value
    EVENTS = events_widget.value
    ATTENDEE_TYPE = attendees_widget.value
    TARGET_METRIC = target_metric_widget.value
    TYPE_OF_DISPLAY = display_type_widget.value
    #outputs graph generated from data analysis dataframe
    result_df = analyze_data(YEARS, QUARTERS, EVENTS, ATTENDEE_TYPE, TARGET_METRIC, TYPE_OF_DISPLAY)
    if result_df.empty:
      # handles if no data is found
      print("No data within specified parameters")
    else:
      result_df.plot(x = TARGET_METRIC, y='# of Students', kind = 'bar')

###############################DATA ANALYSIS CODE###############################
def analyze_data(YEARS, QUARTERS, EVENTS, ATTENDEE_TYPE, TARGET_METRIC, TYPE_OF_DISPLAY):
  #translate TARGET_METRIC to COLUMN_OF_INTEREST
  COLUMN_OF_INTEREST = TARGET_METRIC
  if COLUMN_OF_INTEREST == 'Major/Degree':
    if ATTENDEE_TYPE == 'Undergraduate Students':
      COLUMN_OF_INTEREST = 'Undergraduate Major'
    if ATTENDEE_TYPE == 'Graduate Students':
      COLUMN_OF_INTEREST = 'Degree Program:' 

  # creates empty data frame used to merge all target data into
  merged_target_df = pd.DataFrame()

  #loops through specified search parameters
  for YEAR in YEARS:
    for QUARTER in QUARTERS: 
      for EVENT in EVENTS:
        #sets one search path of interest
        current_path =  'Raw Data/' + YEAR + '/' + QUARTER + '/' + EVENT
        # narrow down files to just excels
        filenames = glob.glob(current_path + '/*.xlsx')
        if filenames != []:
          for file in filenames: 
          # iterates through excel files and append relevant info to merged_target_df
            next_all_df = pd.read_excel(file, sheet_name= 0)
            next_all_df['Full Name'] = next_all_df['First Name'] + next_all_df['Last Name']

            if COLUMN_OF_INTEREST == 'Role at Stanford':
              if 'Which degree(s) are you pursuing?' in next_all_df: 
                COLUMN_OF_INTEREST = 'Which degree(s) are you pursuing?'
              elif 'Role at Stanford' in next_all_df: 
                COLUMN_OF_INTEREST = 'Role at Stanford'
              else:
                break

            # handle exception for Major/Degree metric, which analyzes two different columns for Undergrads and Grads, if specified for All Students
            if TARGET_METRIC == 'Major/Degree' and ATTENDEE_TYPE == 'All Students':
              if 'Degree Program:' in next_all_df and 'Undergraduate Major' in next_all_df:
                COLUMN_OF_INTEREST = 'Major/Degree'
                next_all_df['Major/Degree'] = next_all_df['Undergraduate Major'].fillna('').astype(str) + next_all_df['Degree Program:'].fillna('').astype(str)
              elif 'Undergraduate Major' in next_all_df:
                COLUMN_OF_INTEREST = 'Undergraduate Major'
              else:
                COLUMN_OF_INTEREST = 'Degree Program:'
                
            next_target_df = next_all_df[['Full Name', 'Ticket Type', COLUMN_OF_INTEREST]].copy()
            
            merged_target_df = pd.concat([merged_target_df, next_target_df], ignore_index = True)
  if merged_target_df.empty: 
    return merged_target_df
  else: 
    if TYPE_OF_DISPLAY == 'Unique Visits':
      # filters out duplicate attendee data if specified
      merged_target_df = merged_target_df.drop_duplicates(subset=['Full Name'])

    if ATTENDEE_TYPE != 'All Students':
      # can be used to further filter out data (i.e. by Undergrad vs Grad, etc.), uncomment to use 
      merged_target_df = merged_target_df[merged_target_df["Ticket Type"] == ATTENDEE_TYPE]

    # separate merged column of interest into col_df
    col_df = merged_target_df[COLUMN_OF_INTEREST]

    if TARGET_METRIC == 'Year at Stanford':
      #used when searching Year at Stanford to  strip extra spaces
      col_df = col_df.fillna('No Info').astype(str)
      col_df = col_df.replace(' ', '')
      col_df = col_df.replace('1.0', '1')
      col_df = col_df.replace('2.0', '2')
      col_df = col_df.replace('3.0', '3')
      col_df = col_df.replace('4.0', '4')
      col_df = col_df.replace('5.0', '5+')
    count = col_df.value_counts() ##if count zero
    #turn count Series to df and change column names
    count_df = pd.DataFrame({TARGET_METRIC:count.index, '# of Students':count.values})
    return count_df

#####################CREATES WEB APP PLATFORM###################
# Create interactive widgets to specify parameters
years_widget = widgets.SelectMultiple(  # SelectMultiple widget used for parameters where you can select multiple; use shift to select each option
    options=['2020-2021', '2021-2022'],
    value=['2020-2021'],
    description='ACADEMIC YEARS',
    disabled=False,
    style = {'description_width': 'initial'}
)

quarters_widget = widgets.SelectMultiple(
    options=['Fall', 'Winter', 'Spring'],
    value=['Fall'],
    description='ACADEMIC QUARTERS',
    disabled=False,
    style = {'description_width': 'initial'}
)

events_widget = widgets.SelectMultiple(
    options=['Chem Virtual Study Halls', 'ENG Grad Studios', 'Math Virtual Study Halls', 'Virtual Study Halls', 'Workshops', 'Grad Studios', 'Study Halls', 'Chem 31B Study Halls', 'Studios', 'In-Person Study Halls'],
    value=['Chem Virtual Study Halls'],
    description='CTL EVENT TYPES',
    disabled=False,
    style = {'description_width': 'initial'}
)

attendees_widget = widgets.RadioButtons(  # RadioButton widget used for single selection parameters
    options=['Graduate Students', 'Undergraduate Students', 'All Students'],
    description='STUDENT TYPES',
    disabled=False,
    style = {'description_width': 'initial'}
)

target_metric_widget = widgets.RadioButtons(
    options=['Year at Stanford', 'Major/Degree', 'Stanford School', 'Role at Stanford'],
    description='TARGET INFORMATION',
    disabled=False,
    style = {'description_width': 'initial'}
)

display_type_widget = widgets.RadioButtons(
    options=['Unique Visit', 'All Visits'],
    description='TYPE OF DISPLAY',
    disabled=False,
    style = {'description_width': 'initial'}
)

# formats parameter selection widgets into nice layout and displays them
row_1 = widgets.HBox([years_widget, quarters_widget, events_widget])
row_2 = widgets.HBox([attendees_widget, target_metric_widget, display_type_widget])
param_options = widgets.VBox([row_1, row_2])
display(param_options)

# creates button that generates graph and calls on_button_clicked
run_button = widgets.Button(
    description='Generate Graph'
)
run_button.on_click(on_button_clicked)
display(run_button)



Button(description='Generate Graph', style=ButtonStyle())

In [1]:
#toggle button code
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')